<a href="https://colab.research.google.com/github/buterajacques1/alu-machine_learning/blob/main/Transfer_Learning/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This project applies transfer learning on the Fashion-MNIST dataset using three pre-trained models: VGG16, ResNet50, and InceptionV3. The goal is to classify images of fashion items into one of ten categories.**


***Step 1: Preparing the Dataset***

In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
# Load the Fashion-MNIST dataset
from tensorflow.keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the images to [0, 1] range and convert to 3 channels
x_train = np.stack([x_train]*3, axis=-1)
x_test = np.stack([x_test]*3, axis=-1)

# Resize the images to 32x32
x_train = np.array([np.pad(image, ((2, 2), (2, 2), (0, 0)), mode='constant') for image in x_train])
x_test = np.array([np.pad(image, ((2, 2), (2, 2), (0, 0)), mode='constant') for image in x_test])

# Convert labels to categorical format
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create data generators
train_datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
validation_generator = ImageDataGenerator().flow(x_test, y_test, batch_size=32)


**Step 2: Fine-Tuning the Pre-trained Models**

**VGG16**

In [ ]:
# Load the VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base layers
for layer in vgg_base.layers:
    layer.trainable = False

# Add custom layers on top
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the final model
vgg_model = Model(inputs=vgg_base.input, outputs=predictions)

# Compile the model
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
vgg_model.fit(train_generator, epochs=5, validation_data=validation_generator)


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 928s 494ms/step - loss: 0.9663 - accuracy: 0.6727 - val_loss: 0.6266 - val_accuracy: 0.7844
Epoch 2/5
 798/1875 [===========>..................] - ETA: 7:37 - loss: 0.7992 - accuracy: 0.7113

**ResNet50**

In [5]:
# Load the ResNet50 model
input_tensor = Input(shape=(32, 32, 3))
resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the base layers
for layer in resnet_base.layers:
    layer.trainable = False

# Add custom layers on top
x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the final model
resnet_model = Model(inputs=resnet_base.input, outputs=predictions)

# Compile the model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = resnet_model.fit(train_generator, epochs=5, validation_data=validation_generator)

# Evaluate the model
loss, accuracy = resnet_model.evaluate(validation_generator)
print(f"Validation accuracy: {accuracy*100:.2f}%")
print(f"Validation loss: {loss:.2f}")

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/5
1875/1875 [==============================] - 399s 209ms/step - loss: 0.8732 - accuracy: 0.7028 - val_loss: 0.6302 - val_accuracy: 0.7650
Epoch 2/5
1875/1875 [==============================] - 382s 204ms/step - loss: 0.7036 - accuracy: 0.7434 - val_loss: 0.5777 - val_accuracy: 0.7888
Epoch 3/5
1875/1875 [==============================] - 380s 203ms/step - loss: 0.6777 - accuracy: 0.7509 - val_loss: 0.5653 - val_accuracy: 0.7951
Epoch 4/5
1875/1875 [==============================] - 423s 226ms/step - loss: 0.6599 - accuracy: 0.7566 - val_loss: 0.5488 - val_accuracy: 0.8002
Epoch 5/5
313/313 [==============================] - 47s 151ms/step - loss: 0.5361 - accuracy: 0.8059
Validation accuracy: 80.59%
Validation loss: 0.54


In [8]:
# Load the InceptionV3 model without the top fully connected layers
input_tensor = Input(shape=(256, 256, 3))
inception_base = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the convolutional base
for layer in inception_base.layers:
    layer.trainable = False

# Add custom layers on top
x = inception_base.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the final model
inception_model = Model(inputs=inception_base.input, outputs=predictions)

# Compile the model
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
inception_model.summary()

# Train the model
history = inception_model.fit(train_generator, epochs=5, validation_data=validation_generator)

# Evaluate the model
loss, accuracy = inception_model.evaluate(validation_generator)
print(f"Validation accuracy: {accuracy*100:.2f}%")
print(f"Validation loss: {loss:.2f}")

87910968/87910968 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 127, 127, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization_2 (Bat  (None, 127, 127, 32)         96        ['conv2d[0][0]']              
 chNormalization)                                                                                 
                                                                                                  
 activation (Activation)   

In [11]:
# Evaluate the models
#vgg_result = vgg_model.evaluate(x_test, y_test)
#print(f'VGG16 - Loss: {vgg_result[0]}, Accuracy: {vgg_result[1]}')

resnet_result = resnet_model.evaluate(x_test, y_test)
print(f'ResNet50 - Loss: {resnet_result[0]}, Accuracy: {resnet_result[1]}')

inception_result = inception_model.evaluate(x_test, y_test)
print(f'InceptionV3 - Loss: {inception_result[0]}, Accuracy: {inception_result[1]}')


313/313 [==============================] - 47s 146ms/step - loss: 0.5361 - accuracy: 0.8059
ResNet50 - Loss: 0.5360641479492188, Accuracy: 0.805899977684021


NameError: name 'x_test_inception' is not defined